# Chapter 16

## Load packages / modules

Standard library:
- Printf for string formatting
- Statistics for mean and std functions

External:
- CSV.jl to load CSV files
- DataFrames.jl for tabular data
- CategoricalArrays.jl for working with categorical data columns and the `cut` function
- GLM.jl for models
- Econometrics.jl for instrumental variable 2-stage model
- PyCall.jl for using the Python Statsmodels package
- Roots.jl for finding a zero of a function

In [1]:
using Printf, Statistics
using CSV, DataFrames, CategoricalArrays, GLM, Econometrics, PyCall, Roots

## Load data

In [2]:
nhefs_all = DataFrame(CSV.File("nhefs.csv"));

In [3]:
size(nhefs_all)

(1629, 64)

In [4]:
for s in [:education, :exercise, :active]
    nhefs_all[!, s] = categorical(nhefs_all[!, s])
end

## Section 16.1

Create the instrument, $Z$

In [5]:
nhefs_all.highprice = (nhefs_all.price82 .>= 1.5);

We'll use a different subset of the data than used previously

In [6]:
restriction_cols = [:wt82, :price82]
nhefs = dropmissing(nhefs_all, restriction_cols);

In [7]:
size(nhefs)

(1476, 65)

We'll check whether $Z$ (`highprice`) and $A$ (`qsmk`) are associated, that $\Pr[A=1|Z=1] - \Pr[A=1|Z=0] \not = 0$

In [8]:
a_given_z1 = nhefs.qsmk[nhefs.highprice .== 1]
a_given_z0 = nhefs.qsmk[nhefs.highprice .== 0]

pr_a1_z1 = sum(a_given_z1 .== 1) / size(a_given_z1, 1)
pr_a1_z0 = sum(a_given_z0 .== 1) / size(a_given_z0, 1)

@printf "              Pr[A=1|Z=1] = %4.1f%%\n" pr_a1_z1 * 100
@printf "              Pr[A=1|Z=0] = %4.1f%%\n" pr_a1_z0 * 100
@printf "Pr[A=1|Z=1] − Pr[A=1|Z=0] = %4.1f%%" (pr_a1_z1 - pr_a1_z0) * 100

              Pr[A=1|Z=1] = 25.8%
              Pr[A=1|Z=0] = 19.5%
Pr[A=1|Z=1] − Pr[A=1|Z=0] =  6.3%

## Section 16.2

### Program 16.1

Pg 196: For a dichotomous instrument $Z$ that also meets condition (iv) from section 16.3,

$$
    \text{E}[Y^{a=1}] - \text{E}[Y^{a=0}] = \frac{\text{E}[Y|Z=1] - \text{E}[Y|Z=0]}{\text{E}[A|Z=1] - \text{E}[A|Z=0]}
$$

Pg 197: "We estimated the numerator and denominator of the IV estimand by simply calculating the four sample averages ..."

In [9]:
est_y_given_z1 = mean(nhefs.wt82_71[nhefs.highprice .== 1])
est_y_given_z0 = mean(nhefs.wt82_71[nhefs.highprice .== 0])
est_a_given_z1 = mean(nhefs.qsmk[nhefs.highprice .== 1])
est_a_given_z0 = mean(nhefs.qsmk[nhefs.highprice .== 0]);

In [10]:
@printf "estimated E[Y|Z=1] = %0.3f\n" est_y_given_z1
@printf "estimated E[Y|Z=0] = %0.3f\n" est_y_given_z0
@printf "estimated E[A|Z=1] = %0.3f\n" est_a_given_z1
@printf "estimated E[A|Z=0] = %0.3f" est_a_given_z0

estimated E[Y|Z=1] = 2.686
estimated E[Y|Z=0] = 2.536
estimated E[A|Z=1] = 0.258
estimated E[A|Z=0] = 0.195

In [11]:
estimate = (est_y_given_z1 - est_y_given_z0) / (est_a_given_z1 - est_a_given_z0)
@printf "the usual IV estimate: %0.2f kg" estimate

the usual IV estimate: 2.40 kg

"Equivalently, we could have fit two (saturated) linear models to estimate the differences in the denominator and the numerator..."

In [12]:
numer = lm(@formula(wt82_71 ~ highprice), nhefs)
denom = lm(@formula(qsmk ~ highprice), nhefs);

In [13]:
estimate = coef(numer)[2] / coef(denom)[2]
@printf "the usual IV estimate: %0.2f kg" estimate

the usual IV estimate: 2.40 kg

### Program 16.2

Two stage least squares estimator

First, we'll manually use two models, but this will give the wrong confidence interval

In [14]:
stage_1 = lm(@formula(qsmk ~ highprice), nhefs)

nhefs.A_pred = predict(stage_1, nhefs)

stage_2 = lm(@formula(wt82_71 ~ A_pred), nhefs)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

wt82_71 ~ 1 + A_pred

Coefficients:
──────────────────────────────────────────────────────────────────────
               Coef.  Std. Error     t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────
(Intercept)  2.06816     5.14007  0.40    0.6875   -8.01448    12.1508
A_pred       2.39627    20.0545   0.12    0.9049  -36.9422     41.7347
──────────────────────────────────────────────────────────────────────

In [15]:
estimate = coef(stage_2)[2]
@printf "the 2-stage estimate: %0.2f kg" estimate

the 2-stage estimate: 2.40 kg

"A commonly used rule of thumb is to declare an instrument as weak if the F-statistic from the first-stage model is less than 10"

In [16]:
# using code from GLM's `ftest`
function fstat(model)
    rss = deviance(model)
    tss = nulldeviance(model)

    n = Int(nobs(model))
    p = dof(model) - 2    # -2 for intercept and dispersion parameter
    statistic = ((tss - rss) / rss) * ((n - p - 1) / p)
end

fstat (generic function with 1 method)

In [17]:
@printf "1st-stage F-statistic: %0.2f" fstat(stage_1.model)

1st-stage F-statistic: 0.82

The confidence intervals in the second-stage model aren't quite right

We'll do the two-stage model again, using the IV 2-stage model from `Econometrics.jl`

In [18]:
two_stage = fit(
    EconometricModel, @formula(wt82_71 ~ (qsmk ~ highprice)), nhefs
)

Continuous Response Model
Number of observations: 1476
Null Loglikelihood: -5151.70
Loglikelihood: -5135.82
R-squared: NaN
LR Test: 31.75 ∼ χ²(1) ⟹  Pr > χ² = 0.0000
Formula: wt82_71 ~ 1 + (qsmk ~ highprice)
Variance Covariance Estimator: OIM
──────────────────────────────────────────────────────────────────────
               PE        SE      t-value  Pr > |t|      2.50%   97.50%
──────────────────────────────────────────────────────────────────────
(Intercept)  2.06816   5.08682  0.406573    0.6844   -7.91003  12.0464
qsmk         2.39627  19.8468   0.120739    0.9039  -36.5347   41.3272
──────────────────────────────────────────────────────────────────────

These standard errors are slightly higher than the Python version

In [19]:
est = coef(two_stage)[2]
lo, hi = confint(two_stage)[2, 1], confint(two_stage)[2, 2]

println("         estimate   95% C.I.")
@printf "beta_1    %6.2f   (%0.1f, %0.1f)" est lo hi

         estimate   95% C.I.
beta_1      2.40   (-36.5, 41.3)

### Program 16.3

"The parameters of structural mean models can be estimated via g-estimation"

We'll solve this using the same methods as in Program 14.2. Recall, in that program we searched for a $\psi^\dagger$ that would minimize the coefficient on $H(\psi^\dagger)$. The $\psi^\dagger$ that achieves the minimum is our estimate of the causal effect.

In Program 14.2 we ended with a call to `find_zero` from Roots.jl to do the fine-grained search for $\psi^\dagger$. Here, we'll just go straight to that.

In [20]:
sm = pyimport("statsmodels.api");

In [21]:
function logit_abs_alpha(psi)
    H_of_psi = nhefs.wt82_71 .- psi .* nhefs.qsmk
    y = nhefs.highprice
    X = [ones(nrow(nhefs)) H_of_psi]
    groups = convert(Array{Int}, nhefs.seqn)

    lgt = sm.GLM(y, X, family=sm.families.Binomial())
    res = lgt.fit(cov_type="cluster", cov_kwds=Dict("groups" => groups))

    alpha = res.params[2]

    return abs(alpha)
end;

In [22]:
psi_est = find_zero(logit_abs_alpha, 4.0);

In [23]:
@printf "best estimate: %0.3f" psi_est

best estimate: 2.396

## Section 16.5

### Program 16.4

We'll calculate the IV estimate using a few different cutoffs for `highprice`

First, we'll calculate the "usual" IV estimate using data means

In [24]:
function wald_estimate(Y, A, Z)
    numer = mean(Y[Z .== 1]) - mean(Y[Z .== 0])
    denom = mean(A[Z .== 1]) - mean(A[Z .== 0])
    return numer / denom
end

wald_estimate (generic function with 1 method)

In [25]:
for cutoff in [1.6, 1.7, 1.8, 1.9]
    estimate = wald_estimate(
        nhefs.wt82_71,
        nhefs.qsmk,
        (nhefs.price82 .>= cutoff)
    )
    @printf "cutoff price: \$ %0.2f   estimate: %6.2f kg\n" cutoff estimate
end

cutoff price: $ 1.60   estimate:  41.28 kg
cutoff price: $ 1.70   estimate: -40.91 kg
cutoff price: $ 1.80   estimate: -21.10 kg
cutoff price: $ 1.90   estimate: -12.81 kg


Next we'll re-calculate using 2-stage models, to get confidence intervals

In [26]:
println("         estimate       95% C.I.")
for cutoff in [1.6, 1.7, 1.8, 1.9]
    nhefs.highprice = (nhefs.price82 .>= cutoff)

    model = fit(
        EconometricModel, @formula(wt82_71 ~ (qsmk ~ highprice)), nhefs
    )
    
    est = coef(model)[2]
    lo, hi = confint(model)[2, 1], confint(model)[2, 2]

    @printf "\$ %0.2f     %6.2f   (%6.1f, %6.1f)\n" cutoff est lo hi
end
    
# restore `highprice` to its original meaning
nhefs.highprice = (nhefs.price82 .>= 1.50);

         estimate       95% C.I.
$ 1.60      41.28   (-282.4,  365.0)
$ 1.70     -40.91   (-409.3,  327.5)
$ 1.80     -21.10   ( -76.9,   34.7)
$ 1.90     -12.81   ( -59.3,   33.6)


The standard errors here and in the next program are again slightly different from the Python version

### Program 16.5

In [27]:
spec = @formula(
    wt82_71 ~ sex
            + race
            + age
            + smokeintensity
            + smokeyrs
            + exercise
            + active
            + wt71
            + (qsmk ~ highprice)
)
model = fit(EconometricModel, spec, nhefs)

Continuous Response Model
Number of observations: 1476
Null Loglikelihood: -5151.70
Loglikelihood: -5104.31
R-squared: NaN
LR Test: 94.78 ∼ χ²(11) ⟹  Pr > χ² = 0.0000
Formula: wt82_71 ~ 1 + sex + race + age + smokeintensity + smokeyrs + exercise + active + wt71 + (qsmk ~ highprice)
Variance Covariance Estimator: OIM
────────────────────────────────────────────────────────────────────────────────────
                     PE          SE        t-value  Pr > |t|       2.50%      97.50%
────────────────────────────────────────────────────────────────────────────────────
(Intercept)     17.2803      2.3362      7.39677      <1e-12   12.6977    21.863
sex             -1.64439     2.63173    -0.624833     0.5322   -6.80676    3.51797
race            -0.183255    4.65197    -0.0393929    0.9686   -9.30851    8.942
age             -0.16364     0.24063    -0.680049     0.4966   -0.635656   0.308376
smokeintensity   0.0057669   0.145553    0.0396205    0.9684   -0.279749   0.291283
smokeyrs      